# Desafio 8 - PFSP Cmax Gurobi - Dupla 09

**Integrantes:**

* Lucas Hideki Takeuchi Okamura NUSP: 9274315

* Thales Arantes Kerche Nunes NUSP: 10769372

**Objetivo**

- Encontrar a solução ótima das instâncias do problema de *flow shop* permutacional considerados nos desafios anteriores, incluindo o tempo computacional e o gap de otimalidade.

**Descrição**

1. Implantar o modelo de otimização para PFSP Cmax dado abaixo no Gurobi

2. Testar o modelo com instâncias pequenas

3. Implementar um código para leitura, solução e impressão dos resultados (leitura e impressão em planilha, solução em Gurobi)

**Formulação Matemática**

**Índices**

i - job

j - posição na sequência

k - máquina

**Parâmetros**

m - número de máquinas

n - número de jobs

**p** - lista dos tempos de processamento dos jobs

**Variáveis de Decisão:**

$ x_{ij}=
\begin{cases}
1, & \text{se i está na posição j da sequência} \\
0, & \text{c.c.}
\end{cases} $

$ s_{jk}: \text{start time do job na posição j na máquina k} $

$ c_{jk}: \text{completion time do job na posição j na máquina k} $

**Modelo**

\begin{equation}
\begin{aligned}
& \quad \text{min} \quad c_{nm} \\
&\text{s.a.} \\
& \quad c_{jk} \geq s_{jk} + \sum_{i=1}^{n} x_{ij} p_{ik} \quad j=1,...,n \quad k=1,...,m\\
& \quad s_{jk} \geq c_{j-1,k} \quad j=2,...,n \quad k=1,...,m \\
& \quad s_{jk} \geq c_{j,k-1} \quad j=1,...,n \quad k=2,...,m \\
& \quad \sum_{i=1}^{n} x_{ij} = 1 \quad j=1,...,n \\
& \quad \sum_{j=1}^{n} x_{ij} = 1 \quad i=1,...,n \\
& \quad \quad \quad s_{jk},c_{jk} \geq 0, x_{ij} \in \{ 0,1 \} \\
\end{aligned}
\end{equation}

In [1]:
import xlwings as xw
import numpy as np
import gurobipy as grb

## 1. Definindo a Classe Job

In [2]:
class Job:
    def __init__(self,i,p):
        self.i=i     # número do job, pela ordem de chegada
        self.p=p     # processing times [p1,p2]
        self.C=[]     # completion times [c1,c2]
        self.psum = np.sum(self.p)     # sum of values of p

## 2. Definindo Funções

**Função para o cálculo dos tempos de conclusão dos jobs em cada uma das m máquinas**

$p_{(j)k}:$ processing time do job da posição j na máquina k

$c_{(j)k}:$ completion time do job da posição j na máquina k

$c_{(1)1} = p_{(1)1}$

$c_{(1)k}=c_{(1),k−1}+p_{(1)k}$, onde    $k=2,...,m$

$c_{(j)1}=c_{(j−1)1}+p_{(j)1}$, onde $j=2,...,n$

$c_{(j)k}=max\left\{{c_{(j),k−1},c_{(j−1)k}}\right\} +p_{(j)k}$, onde $j=2,...,n$ e $k=2,...,m$

In [3]:
def calcCmax(m,jobs):     # esta função calcula os tempos de conclusão dos jobs nas m máquinas
    n=len(jobs)
    c=[[0 for i in range(m)] for j in range(n)]
    p=[job.p for job in jobs]
    
    # primeiro job
    c[0][0]=p[0][0]     # primeira máquina
    for k in range(1,m):     # próximas máquinas
        c[0][k]=c[0][k-1]+p[0][k]
        
    # próximos jobs
    for j in range(1,n):
        c[j][0]=c[j-1][0]+p[j][0]     # primeira máquina
        for k in range(1,m):     # próximas máquinas
            c[j][k]=max(c[j][k-1],c[j-1][k])+p[j][k]
            
    j=0
    for job in jobs:
        job.C=c[j]
        j+=1
    
    return c[-1][-1]

**Entrada de Dados**

In [4]:
def leInst(i):
    plan = wb1.sheets[i]
    n=int(plan.range('B6').value)
    m=int(plan.range('B5').value)
    tab=plan.range('A11').expand('table').value
    jobs=[]
    for j in range(n):
        p=tab[j][1:m+1]
        jobs.append(Job(tab[j][0],p))
        
    #print()
    #print('#','p')
    #for job in jobs:
    #    print(job.i, job.p)
    
    return jobs, m, n

**Saída de resultados**

In [5]:
def gravaSched(i,jobs,Cmax,runtime,gap):
    n=len(jobs)
    tab=[]
    for job in jobs:
        a=[job.i]+job.p+job.C
        tab.append(a)

    if i==0:
        plan=wb2.sheets[0]
        plan.name='I(1)'
    else:
        plan=wb2.sheets.add('I('+str(i+1)+')',after=i)
        
    plan.range('A5').value=['m',m]
    plan.range('A6').value=['n',n]
    plan.range('E5').value=['Cmax',Cmax]
    plan.range('E6').value=['Runtime',runtime]
    plan.range('E7').value=['Optimaly gap',gap]
    plan.range('A10').value=['#','p1','p2','p3','p4','p5','C1','C2','C3','C4','C5']
    plan.range('A11').value=tab

    print()
    # print('#','p','C')
    # for job in jobs:
    #    print(job.i, job.p, job.C)
    print('Cmax:',Cmax)

In [6]:
def schedGurobi(jobs, m, n):
    mPFSP = grb.Model(name="Permutation Flowshop Problem")

    # Definindo Variáveis
    x = mPFSP.addVars(n, n, vtype=grb.GRB.BINARY, name='x')
    s = mPFSP.addVars(n, m, vtype=grb.GRB.CONTINUOUS, lb = 0, name='s')
    c = mPFSP.addVars(n, m, vtype=grb.GRB.CONTINUOUS, lb = 0, name='c')
    mPFSP.update()
    
    # Definindo Contraints
    r1 = mPFSP.addConstrs((c[j, k] - s[j, k] - grb.quicksum(jobs[i].p[k]*x[i,j] for i in range(n)) >= 0 for k in range(m) for j in range(n)), name='r1')
    r2 = mPFSP.addConstrs((s[j, k] - c[j - 1, k] >= 0 for k in range(m) for j in range(1, n)), name = 'r2')
    r3 = mPFSP.addConstrs((s[j, k] - c[j , k - 1] >= 0 for k in range(1, m) for j in range(n)), name = 'r3')
    r4 = mPFSP.addConstrs((grb.quicksum(x[i, j] for i in range(n)) == 1 for j in range(n)), name = 'r4')
    r5 = mPFSP.addConstrs((grb.quicksum(x[i, j] for j in range(n)) == 1 for i in range(n)), name = 'r5')
    mPFSP.update()
    
    # Definindo métrica de otimização
    mPFSP.ModelSense = grb.GRB.MINIMIZE
    mPFSP.setObjective(c[n - 1, m - 1])
    mPFSP.update()
    
    # Obtendo resultados
    mPFSP.optimize()
    runtime = mPFSP.Runtime
    gap = mPFSP.MIPGap
    mvars = mPFSP.getVars()
    
    jobs_final = list(range(n))
    for i in range(n):
        for j in range(n):
            if i == 0:
                if mvars[i*n + j].x == 1:                  # mvars tem essa indexacao de uma lista só
                    jobs_final[j] = jobs[i]                # diz que o iésimo job deve estar na posição j
            else:
                if mvars[i*n + j].x == 1:
                    jobs_final[j] = jobs[i]
                    
    mPFSP.reset(clearall = 1)          # reseta o modelo para outra instancia
                
    return jobs_final, runtime, gap

## 3. Testando com Dados do Excel

In [7]:
ps = [[16, 18, 12],
      [14, 10, 11],
      [13, 20, 15],
      [19, 15, 19],
      [15, 16, 16]]

jobs_test = []
n = len(ps)
m = len(ps[0])

for i in range(n):
    jobs_test.append(Job(i, ps[i]))
    
jobs_test, runtime, opt_gap = schedGurobi(jobs_test, m, n)
bestCmax = calcCmax(m, jobs_test)

print('#','p','C')
for job in jobs_test:
    print(job.i, job.p, job.C)
print('Cmax:',bestCmax)
print('Runtime:', runtime, "segundos")
print('Optimaly Gap:', opt_gap)

Using license file C:\Users\LHTO\gurobi.lic
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 47 rows, 55 columns and 199 nonzeros
Model fingerprint: 0x34c1ce15
Variable types: 30 continuous, 25 integer (25 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 14 rows and 15 columns
Presolve time: 0.00s
Presolved: 33 rows, 40 columns, 153 nonzeros
Variable types: 10 continuous, 30 integer (25 binary)
Found heuristic solution: objective 115.0000000
Found heuristic solution: objective 112.0000000
Found heuristic solution: objective 109.0000000

Root relaxation: objective 1.043698e+02, 41 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  104.36981  

In [8]:
wb1 = xw.Book('xl05 2 A PFSP Cmax CDS.xlsx')
wb2 = xw.Book()
I = wb1.sheets.count     # número de instância é igual ao número de planilhas na pasta

for i in range(I):
    print()
    print(i+1)
    jobs, m, n = leInst(i)
    jobs, runtime, gap = schedGurobi(jobs, m, n)
    Cmax = calcCmax(m, jobs)
    gravaSched(i,jobs,Cmax,runtime,gap)
    
wb2.save('xl12 2 B PFSP Cmax Gurobi.xlsx')


1
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 315 rows, 600 columns and 3350 nonzeros
Model fingerprint: 0x0e95097f
Variable types: 200 continuous, 400 integer (400 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 48 rows and 49 columns
Presolve time: 0.02s
Presolved: 267 rows, 551 columns, 3135 nonzeros
Variable types: 129 continuous, 422 integer (400 binary)
Found heuristic solution: objective 1379.0000000
Found heuristic solution: objective 1369.0000000

Root relaxation: objective 1.171000e+03, 883 iterations, 0.04 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1171.00000    0   60 1369.00000 1171.00000  14.5%     -    0s
H    0     0                    1280.0000000 1171.00000  8.52%     -   


Root relaxation: objective 1.138200e+03, 682 iterations, 0.03 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1138.20000    0   51 1348.00000 1138.20000  15.6%     -    0s
H    0     0                    1318.0000000 1138.20000  13.6%     -    0s
H    0     0                    1280.0000000 1138.20000  11.1%     -    0s
H    0     0                    1267.0000000 1138.20000  10.2%     -    0s
H    0     0                    1261.0000000 1146.55106  9.08%     -    0s
H    0     0                    1252.0000000 1146.55106  8.42%     -    0s
H    0     0                    1222.0000000 1147.00000  6.14%     -    0s
     0     0 1147.00000    0   34 1222.00000 1147.00000  6.14%     -    0s
H    0     0                    1194.0000000 1147.00000  3.94%     -    0s
H    0     0                    1177.0000000 1147.00000  2.55%     -    0s
H    0     0               

Found heuristic solution: objective 1514.0000000

Root relaxation: objective 1.290000e+03, 419 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1290.00000    0   18 1514.00000 1290.00000  14.8%     -    0s
H    0     0                    1369.0000000 1290.00000  5.77%     -    0s
H    0     0                    1308.0000000 1290.00000  1.38%     -    0s
H    0     0                    1299.0000000 1290.00000  0.69%     -    0s
     0     0 1290.00000    0   23 1299.00000 1290.00000  0.69%     -    0s
     0     0 1290.00000    0   27 1299.00000 1290.00000  0.69%     -    0s
H    0     0                    1293.0000000 1290.00000  0.23%     -    0s
     0     0 1290.00000    0   27 1293.00000 1290.00000  0.23%     -    0s
H    0     0                    1292.0000000 1290.00000  0.15%     -    0s
H    0     0                    1290.0000000 1290.000